In [1]:
!nvidia-smi

Fri Aug 13 13:34:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd drive/MyDrive/knlp/code

/content/drive/MyDrive/knlp/code


In [3]:
%%capture
!pip install transformers
!pip install wandb
!pip install madgrad
# !pip install keybert

In [5]:
!wandb login $wandb_key

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Training

In [6]:
import main

In [7]:
!ls ../data

EDA.ipynb	  goal.csv		 test.csv
extra_label.csv   labels_mapping.csv	 test_gce_keywords.csv
gce_keywords.csv  sample_submission.csv  train.csv


In [8]:
from setting import Setting
is_small_test = False

setting = Setting(
    data_file='/content/drive/My Drive/knlp/data/gce_keywords.csv',
    test_file='/content/drive/My Drive/knlp/data/test_gce_keywords.csv',
    labels_mapping='/content/drive/My Drive/knlp/data/labels_mapping.csv',
    extra_label_file='/content/drive/My Drive/knlp/data/extra_label.csv',

    model_name='klue/bert-base',
    model_type='mixed', # mixed default
    batch_size=32,
    num_workers=2,
    epochs=10,
    lr = 2e-5,
    criterion='CrossEntropyLoss', # CrossEntropyLoss F1_Loss
    optimizer='MADGRAD', # AdamW MADGRAD
    lr_scheduler='get_cosine_schedule_with_warmup',
    warmup_ratio=0.3,
    madgrad_weight_decay=1e-5,
    warmup_step=0,
    small_data=is_small_test,
    use_amp=True,
    logging= not is_small_test,
    test_ratio=0,
    device_pref='cuda',
    project='',
    entity='',
    name='mixed_bb6_all_data',
    tags=['v0', 'mixed', 'madgrad'],
    # v0 = 처음 세팅
    # v1 = no_empty_label 적용
    # v2 = line stuff
    # v3 = f1_loss
    # v4 = balanced weight in loss
    # v5 = extra label 

    is_testing=False,
    no_empty_label=False,
    single_col_data='과제명',

    freezing_layer_num=-1,
    freeze_emb=False,

    gate_th = 0.5,
    gate_44 = True,
    gate_44_th = 0.9,
    big_mixed_use_each = False,

    use_swa = False,
    swa_lr = 1e-4,
    swa_annel_steps_ratio = 0.1,
    swa_annel_steps = 3,
    swa_start_epoch = 7,

    tokenize_max_seq = 500,

    line_stuff = False,
    line_file = '/content/drive/My Drive/knlp/data/goal.csv',

    clean_dirty = False,
    special_tms = 64,

    mixed_col_keys_to_use = [
                             '사업명', 
                             '사업_부처명', 
                             '내역사업명', 
                             '과제명', 
                             '요약문_한글키워드', 
                             '요약문_영문키워드',
                            #   'goal_keywords',
                            #   'content_keywords',
                            #   'effect_keywords',
                       ],

    use_weight_in_loss=False,
    use_extra_label=False,
    el_single_output=False,

    use_llrd = False,
    llrd_decay = 0.95,

    multi_sample_dropout_n=1,
    output_dropout_p=0.1,
)


In [1]:
result = main.main(setting)

# Predict Test

In [10]:
from test_stuff import test_main
from utils import get_unique_name
from google.colab import files

if not setting.is_testing:
    answer_df = test_main(setting, result)
    saving_file_name = get_unique_name('submission_files', setting.name, extension='csv')
    answer_df.to_csv(saving_file_name, index=False)

    files.download(saving_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Save

In [11]:
# val_df = result['val_dl'].dataset.df

# val_df.to_csv('/content/drive/My Drive/knlp/val.csv', index=False)

In [12]:
# import pandas as pd
# val_df = pd.read_csv('/content/drive/My Drive/knlp/val.csv')

In [13]:
# import pickle

# with open('/content/drive/My Drive/knlp/setting.pkl', 'wb') as f:
#     pickle.dump(setting, f)

# with open('/content/drive/My Drive/knlp/setting.pkl', 'rb') as f:
#     setting = pickle.load(f)

In [14]:
# import torch

# model = result['model']

# torch.save(model.state_dict(), '/content/drive/My Drive/knlp/model.pth')

In [15]:
# from dataset import DataFrameTwoDataset
# from torch.utils.data import DataLoader
# from model import get_model, get_tokenizer

# model = get_model(setting)
# model.load_state_dict(torch.load('/content/drive/My Drive/knlp/model.pth'))
# model.eval()

# tokenizer = get_tokenizer(setting.model_name)
# print()

# val_ds = DataFrameTwoDataset(setting, val_df, tokenizer, is_test=True)

# val_dl = DataLoader(val_ds, batch_size=setting.batch_size, collate_fn=val_ds.collate_fn,
#             shuffle=False, drop_last=False, num_workers=setting.num_workers,
#             pin_memory=True)

# from test_stuff import run_test

# model.cuda()
# preds = run_test(setting, model, val_dl)

# val_df['pred'] = preds
# val_df.head(2)

In [16]:
# val_df.to_csv('/content/drive/My Drive/knlp/val_pred.csv', index=False)

In [17]:
# import pickle
# import torch

# val_df = result['val_dl'].dataset.df

# val_df.to_csv('/content/drive/My Drive/knlp/val.csv', index=False)

# with open('/content/drive/My Drive/knlp/setting.pkl', 'wb') as f:
#     pickle.dump(setting, f)

# model = result['model']

# torch.save(model.state_dict(), '/content/drive/My Drive/knlp/model.pth')

# KeyBert

In [18]:
# %%capture
# !pip install keybert
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [19]:
def key_bert_stuff(df_name, key_name, col_name, merged_save_name):
    from keybert import KeyBERT

    kw_model = KeyBERT(model='paraphrase-multilingual-mpnet-base-v2')

    import warnings
    import pandas as pd
    warnings.filterwarnings('ignore')

    train_df = pd.read_csv(df_name)
    train_df = train_df.fillna('없음')
    train_df.loc[train_df[key_name]  ==  '0', key_name] = '없음'
    train_df.loc[train_df[key_name]  ==  0, key_name] = '없음'

    from tqdm.auto import tqdm, trange

    total = []
    batch_size = 2
    rep = (len(train_df) + batch_size - 1) // batch_size
    for idx in trange(rep):
        who = train_df['index'].iloc[batch_size*idx:batch_size*(idx+1)].tolist()
        lines = train_df[key_name].iloc[batch_size*idx:batch_size*(idx+1)].tolist()
        keywords = kw_model.extract_keywords(lines, keyphrase_ngram_range=(1, 5),
                                    highlight=False, use_maxsum=False, top_n=1)
        keywords = [b[0][0] for b in keywords]
        total.extend(zip(who, keywords))

    del kw_model
    del train_df

    import gc

    gc.collect()

    import pandas as pd

    tmp_dict = {}
    for idx, rows in tqdm(total):
        tmp_dict[idx] = {'index': idx, col_name: rows}

    keywords_df = pd.DataFrame.from_dict(tmp_dict, orient='index')
    train_df = pd.read_csv(df_name)
    goal_merged_df = train_df.merge(keywords_df, on='index')
    goal_merged_df.to_csv(merged_save_name, index=False)

    del goal_merged_df
    gc.collect()

In [20]:

def spacing_stuff(setting, df_name, new_col_name, merged_save_name):
    import gc
    import pandas as pd
    df = pd.read_csv(df_name)

    from model import get_tokenizer

    tokenizer = get_tokenizer(setting.model_name)
    tok = tokenizer(df[new_col_name].tolist(), return_length=True)

    limit = 40
    import numpy as np
    a = np.array(tok['length'])
    xx = list(zip(df['index'], tok['length']))

    aa = np.array(xx)
    long_index = (aa[:, 1] >= limit)

    if long_index.sum() == 0:
      return df

    from  pykospacing  import  Spacing

    spacing = Spacing()


    from tqdm.auto import tqdm

    tmp = {}
    for idx in tqdm(aa[long_index, 0]):
        x = df.loc[df['index'] == idx, col_name].item()

        spaced = spacing(x)
        tmp[idx] = {
            'index': idx,
            'spaced': spaced,
        }

    s = pd.DataFrame.from_dict(tmp, orient='index')

    from keybert import KeyBERT

    kw_model = KeyBERT(model='paraphrase-multilingual-mpnet-base-v2')

    import warnings
    import pandas as pd

    warnings.filterwarnings('ignore')

    from tqdm.auto import tqdm, trange

    total = []

    for idx in s['index']:
        line = s.loc[s['index'] == idx, space_name].item()
        keywords = kw_model.extract_keywords(line, keyphrase_ngram_range=(1, 5),
                                    highlight=False, use_maxsum=False, top_n=1)[0][0]
        
        total.append((idx, keywords))

    import pickle


    import pandas as pd

    tmp_dict = {}
    for idx, rows in tqdm(total):
        df.loc[df['index'] == idx, new_col_name] = rows

    df.to_csv(merged_save_name, index=False)

    del df
    gc.collect()

In [21]:
import gc
run_key_extract = False

if run_key_extract:
    names = [('요약문_연구목표', 'goal_keywords'), ('요약문_연구내용', 'content_keywords'), ('요약문_기대효과', 'effect_keywords')]
    # df_name = setting.test_file
    
    df_name = '/content/drive/My Drive/knlp/data/not_converted.csv'
    merged_save_name = '/content/drive/My Drive/knlp/data/fixed.csv'

    for key_name, col_name, in names:
        print('key_name', key_name)
        print('key_bert_stuff')
        merged_df = key_bert_stuff(df_name, key_name, col_name, merged_save_name)
        gc.collect()

        df_name = merged_save_name

        print('key_name', key_name)
        print('spacing_stuff')
        spacing_stuff(setting, df_name, col_name, merged_save_name)
        gc.collect()
